This notebook fetches and formats data from a number of sources: 

* Farm and garden data prepared in _urban_agriculture.ipynb_
* 2018 NYC PLUTO building classifications from NYC DCP
* Income, education, and other demographic variables from ACS via Data2Go
* Violent crime data from NYPD
* Voter participation rates from the Campaign Finance Board
* Census response rates from Census.gov
* Reported mental and physical health from CDC PLACES
* Census tract shapefiles and populations centroids from Census.gov

aggregating point measures to the census tract level and joining all for modeling in _non_spatial_regression.ipynb_


Link to Drive and path to data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd '/content/drive/My Drive/Summer/MAP NYC x CUSP/data/social_infrastructure'

/content/drive/My Drive/Summer/MAP NYC x CUSP/data/social_infrastructure


Install and import required packages

In [3]:
!pip install geopandas rtree

     |████████████████████████████████| 1.0 MB 29.4 MB/s 
     |████████████████████████████████| 994 kB 48.3 MB/s 
     |████████████████████████████████| 15.4 MB 39 kB/s 
     |████████████████████████████████| 6.3 MB 56.7 MB/s 


In [42]:
import requests
import numpy as np
import pandas as pd
import geopandas as gpd

import requests
from urllib.request import urlretrieve
from zipfile import ZipFile
from os import remove as remove_file
from shutil import rmtree as remove_dir

Define API handler for Open Data sources

In [5]:
def get_socrata(resource, query='', dates=[], limit=1000, sample=False, domain='data.cityofnewyork.us', progress=False):
    headers = {'X-App-Token': 'b0nu5h6f5RSJRNPajPuuCf7wo'}
    result = []
    offset = 0
    while offset >= 0:
        url = f'https://{domain}/resource/{resource}.json?$limit={limit}&$offset={offset}&$order=:id'
        if query != '': #can't use requests params bc of possible $ char
            url += f'&{query}'
        if (progress):
            print(url)
        json = requests.get(url, headers=headers).json()
        if 'error' in json:
            print(json['message'])
            return
        else:
            df = pd.DataFrame(json)
            if len(df):
                if len(dates):
                    df[dates] = df[dates].apply(pd.to_datetime, errors='coerce')
                result.append(df)
                offset += limit
            else:
                offset = -1
        if sample:
          break

    result = pd.concat(result).reset_index() if len(result) else pd.DataFrame()
    print(f'Retreived {len(result)} records')
    return result

Load the _urban_ag.csv_ output from _urban_ag_formatter.ipynb_

In [6]:
usecols = ['name', 'address', 'type', 'grow', 'borocode', 'block', 'lot', 'lat', 'lon', 'lotarea']
urban_ag = pd.read_csv('../urban_agriculture/urban_ag.csv', usecols=usecols)
urban_ag.columns = urban_ag.columns.str.lower()
urban_ag.head()

,name,address,type,grow,lat,lon,lotarea,borocode,block,lot
0,Fishbridge Park Garden,"Pearl Street, New York, NY 10038, USA",Community Garden,Outdoor,40.709068,-74.001378,4034.0,1.0,106.0,22.0
1,Lower East Side People Care,"25 Rutgers Street, New York, NY 10002, USA",Community Garden,Outdoor,40.713006,-73.989824,2135.0,1.0,271.0,63.0
2,Clinton Community Garden (LES),"Stanton Street, New York, NY 10002, USA",Community Garden,Outdoor,40.720010,-73.984459,2533.0,1.0,344.0,148.0
3,Siempre Verde Garden,"Stanton Street, New York, NY 10002, USA",Community Garden,Outdoor,40.719930,-73.984019,1197.0,1.0,344.0,153.0
4,Siempre Verde Garden,"Attorney Street, New York, NY 10002, USA",Community Garden,Outdoor,40.719744,-73.984008,1405.0,1.0,344.0,158.0


Aggregate by name, borough and block (many gardens are in adjacent lots and should be counted as a single garden)

In [7]:
x = urban_ag['type'] == 'NYCHA Garden'
urban_ag.loc[x, 'name'] = urban_ag.loc[x, 'name'].str.replace(' Garden [0-9]+$', '') # aggregate NYCHA gardens by development

In [8]:
urban_ag['borocode'].fillna(0, inplace=True)
urban_ag['block'].fillna(0, inplace=True)
urban_ag['grow'].fillna('', inplace=True)

ag_unique = (
    urban_ag
    .groupby(['name', 'type', 'grow', 'borocode', 'block'])
    .agg({'lat': 'mean', 'lon': 'mean', 'address': 'first', 'lotarea': 'sum'}, min_count=1)
    .reset_index()
)
ag_unique['type'].value_counts()

School Garden                 676
Community Garden              583
NYCHA Garden                  276
Potential Community Garden     74
Non-commercial Farm            29
Commercial Farm                22
Name: type, dtype: int64

Define some dictionary maps and variable for subsequent formatting

In [9]:
boro_counties = {'1': '061', '2': '005', '3': '047', '4': '081', '5': '085'}

counties_str = list(boro_counties.values())
counties = [int(x) for x in counties_str]

type_codes = {'Commercial Farm': 'FC',
              'Non-commercial Farm': 'FN',
              'Community Garden': 'GC',
              'NYCHA Garden': 'GN',
              'School Garden': 'GS',
              'Potential Community Garden': 'GP'}
types = list(type_codes.keys())

In [10]:
ag_unique.groupby('type')['lotarea'].quantile(.75)

type
Commercial Farm               123399.75
Community Garden               10543.50
NYCHA Garden                    4595.25
Non-commercial Farm           145000.00
Potential Community Garden      7798.75
School Garden                 108400.25
Name: lotarea, dtype: float64

Cap the lot size of certain farms of known size and others based on distribution of sizes for each type (tax lots can be significantly larger than the farm/garden itself)

In [11]:
ag_unique['area_upper'] = (
    np.where(ag_unique['name'] == 'Queens County Farm', 200000, 
             np.where(ag_unique['name'] == 'Brooklyn Grange Rooftop Farms', 135000,
                      np.where(ag_unique['name'] == 'GrowNYC Teaching Garden at Governors Island', 4000, #listed as garden, not farm
                               np.where(ag_unique['type'].str.contains('Farm'), 40000, # ~1 acre
                                        np.where(ag_unique['type'].str.contains('Community'), 10000, #~75th percentile
                                                 2000))))) # NYCHA 75th is probably the most realistic for schools too
 )
ag_unique['area'] = ag_unique['lotarea'].clip(upper=ag_unique['area_upper'])
ag_unique['typecode'] = ag_unique['type'].map(type_codes)

Make GeoDataFrames for spatial joins

In [12]:
ll_crs = 'EPSG:4326'
nyc_crs = 'EPSG:2263' # DCP equal area proj

def gdf_ll(df, lon='longitude', lat='latitude', crs=ll_crs):
  df = df.dropna(subset=[lon, lat])
  return gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[lon], df[lat]), crs=crs).to_crs(nyc_crs) 

ag_geo = gdf_ll(ag_unique, 'lon', 'lat')

Load PLUTO data for building classifications and make sure relevant lots are distinct from urban agriculture dataset

In [43]:
ver = '18v2_1'
url = f'https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_{ver}_csv.zip'

tmp = 'tmp/'
zip = 'tmp.zip'
urlretrieve(url, zip)
with ZipFile(zip, 'r') as zip_ref:
    zip_ref.extractall(tmp)

usecols = ['borocode', 'block', 'lot', 'bldgclass', 'xcoord', 'ycoord', 'lotarea']
pluto = pd.read_csv(f'{tmp}pluto_{ver}.csv', usecols=usecols)

remove_dir(tmp)
remove_file(zip)

pluto.head()

,block,lot,bldgclass,lotarea,borocode,xcoord,ycoord
0,5641.0,670.0,T2,53758.0,2.0,1044510.0,246734.0
1,3935.0,3.0,A1,10870.0,4.0,1025679.0,227096.0
2,4205.0,41.0,V0,2369.0,2.0,1026345.0,249922.0
3,2402.0,14.0,G7,3750.0,2.0,1007336.0,238351.0
4,63.0,83.0,A1,3465.0,5.0,958845.0,172482.0


In [44]:
pluto_social = pluto[pluto['bldgclass'].str.contains('^M|^P[5,7-8]|^Q[0-4,7]|^W1|^V[0-6]') == True].copy() #public spaces only; exclude gardens, use our count

social_geo = gdf_ll(pluto_social, 'xcoord', 'ycoord', crs=nyc_crs)
social_geo['ll'] = social_geo['geometry'].to_crs(ll_crs)
social_geo['lon'] = social_geo['ll'].apply(lambda p: p.x)
social_geo['lat'] = social_geo['ll'].apply(lambda p: p.y)

# remove buildings already in ag dataset based on buffer area (BBL is unreliable)
social_geo['geometry'] = social_geo['geometry'].buffer(50) # in ft
social_geo = social_geo.overlay(ag_geo, how='difference')

Define social infrastructure categories

In [15]:
social_unique = (
    social_geo
    .groupby(['borocode', 'block', 'bldgclass'])
    .agg({'lat': 'mean', 'lon': 'mean', 'lotarea': 'sum'}, min_count=1)
    .reset_index()
)
social_unique['bldgcat'] = 'pluto_' + social_unique['bldgclass'].str[0:1].map({'M':'church', 'P':'cultural', 'Q':'outdoor', 'W':'school', 'V':'vacant'})
social_unique['bldgcat'].value_counts()

pluto_vacant      11829
pluto_church       4664
pluto_outdoor      1800
pluto_school       1126
pluto_cultural      542
Name: bldgcat, dtype: int64

Load penal law codes in order to classify reported crimes as violent

In [16]:
pd_code = pd.read_excel('penal_codes.xlsx')
violent_codes = ",".join(pd_code['Violent'].dropna().astype(str))

Fetch NYPD crime complaints. This takes a while, so cache in Drive

In [17]:
# query = f"$where=date_extract_y(cmplnt_fr_dt) in (2016, 2017, 2018) and law_cat_cd = 'FELONY' and ky_cd IN ({violent_codes})"
# crime = get_socrata('qgea-i56i', query, dates=['cmplnt_fr_dt'])
# crime['year'] = crime['cmplnt_fr_dt'].dt.year
# crime = crime[['year', 'latitude', 'longitude']]

# crime.to_csv('violent_crime.csv', index=False)

crime = pd.read_csv('violent_crime.csv')

In [46]:
crime = crime[['year', 'latitude', 'longitude']]

crime.to_csv('violent_crime2.csv', index=False)

Fetch CDC PLACES health data

In [18]:
c = "','".join('36' + pd.Series(counties_str))
places = get_socrata(resource='yjkw-uj5s', domain='chronicdata.cdc.gov', query=f"$where=countyfips IN ('{c}')")
places = places[['tractfips', 'phlth_crudeprev', 'mhlth_crudeprev']]

cols = ['cdc_physical', 'cdc_mental']
places.columns = ['geoid', *cols]
places[cols] = places[cols].astype(float)

Retreived 2117 records


Fetch voter participation data and aggregate 2017 (mayoral election) active voter counts by census tract. This also takes a while, so cache

In [20]:
# vote = get_socrata('psx2-aqx3', query='$where=GE17 == 1')
# vote.to_csv('vote.csv', index=False)

# vote_ct = vote.groupby(['borocode_17', 'ct2010_17'])[['random_id']].count().reset_index()
# vote_ct['county'] = vote_ct['borocode_17'].map(boro_counties)
# vote_ct['geoid'] = '36' + vote_ct['county'].str.zfill(3) + vote_ct['ct2010_17'].str.zfill(6)
# vote_ct.rename(columns={'random_id': 'cfb_votes'}, inplace=True)

# vote_ct.to_csv('vote_ct.csv', index=False)

vote_ct = pd.read_csv('vote_ct.csv')

Fetch census response rates

In [22]:
response = pd.read_json('https://api.census.gov/data/2010/dec/responserate?get=NAME,GEO_ID,FSRR2010&for=tract:*&in=state:36')
response.columns = response.iloc[0]
response = response[1:]
response = response[response['county'].isin(counties_str)]
response['geoid'] = response['GEO_ID'].apply(lambda x : x.split('US')[1])
response['census_response'] = response['FSRR2010'].astype(float)

Load EPA walkabililty scores

In [23]:
walk = pd.read_csv('walkability_nyc.csv', usecols=['GEOID10', 'NatWalkInd'])
walk.columns = ['geoid', 'epa_walk']
walk['geoid'] = walk['geoid'].astype(str).str[:-1]
walk = walk.groupby('geoid').mean().reset_index() # agg from block group level

Load Data2Go (ACS) socioeconomic data

In [24]:
columns = {'GEO_ID': 'geoid',
           'GEO_LABEL': 'geolabel',
           'total_population_tract': 'population',
           'median_household_income_tract': 'acs_income',
           'labor_force_participation_tract': 'acs_participation',
           'income_inequality_tract': 'acs_gini',
           'high_cost_h_tract': 'acs_housing',
           'health_insurance_uninsured_tract': 'acs_uninsured',
           'at_least_bachelors_tract': 'acs_bachelors',
           'at_least_hs_tract': 'acs_hs',
           'white_pop_tract': 'acs_white',
           'foreign_tract': 'acs_foreign',
           'lonely_tract': 'acs_alone',
           'familychild_tract': 'acs_child',
           'rental_vac_tract': 'acs_vacancy',
           'commute_60_mins_plus_tract': 'acs_commute60'}

d2g = pd.read_csv('d2g_5_tract.csv', usecols=list(columns.keys()))
d2g.rename(columns=columns, inplace=True)

d2g = d2g[list(columns.values())]
d2g = d2g.dropna()
d2g['geoid'] = d2g['geoid'].astype(str)

Load census tract boundaries and join to D2G

In [25]:
tract = gpd.read_file('ny_2018_tract').to_crs('EPSG:2263')
tract.columns = tract.columns.str.lower()
tract['area'] = tract.geometry.area

census = d2g.merge(response[['geoid','census_response']]).merge(tract[['geoid', 'area']], on='geoid')
census['density'] = census['population'] / census['area'] # per sqft

Load 2010 census tract population centroids (2020 not yet available)

In [26]:
pop_tract = pd.read_csv('https://www2.census.gov/geo/docs/reference/cenpop2010/tract/CenPop2010_Mean_TR36.txt')
pop_tract.columns = pop_tract.columns.str.lower()
pop_nyc = pop_tract[pop_tract['countyfp'].isin(counties)].copy()

Format population, crime, urban agriculture and social infrastructure data (all lat-lon based) as GeoDFs and aggregate to census tract level (based on population centroid)

In [27]:
pop_geo = gdf_ll(pop_nyc)
pop_geo['geoid'] = pop_geo['statefp'].astype(str).str.zfill(2) + pop_geo['countyfp'].astype(str).str.zfill(3) + pop_geo['tractce'].astype(str).str.zfill(6)
pop_geo['geometry'] = pop_geo['geometry'].buffer(5280 / 2) # in ft

In [ ]:
crime_geo = gdf_ll(crime)
pop_crime = gpd.sjoin(pop_geo, crime_geo)
crime_total = pd.pivot_table(pop_crime, index='geoid', columns='year', values='index_right', aggfunc='count').fillna(0)
crime_total['nypd_violent'] = crime_total.mean(axis=1)
crime_total = crime_total.reset_index()

In [29]:
pop_ag = gpd.sjoin(pop_geo, ag_geo)
ag_total = pd.pivot_table(pop_ag, index='geoid', columns='typecode', values='lotarea', aggfunc='count').fillna(0) # can try sum(lotarea) too
ag_total['nc_agriculture'] = ag_total[['FN', 'GC', 'GN']].sum(axis=1)
ag_total = ag_total.reset_index()

In [30]:
social_geo = gdf_ll(social_unique, 'lon', 'lat')
pop_social = gpd.sjoin(pop_geo, social_geo)
bldg_total = pd.pivot_table(pop_social, index='geoid', columns='bldgcat', values='lotarea', aggfunc='count').fillna(0)
bldg_total = bldg_total.reset_index()

Join all the datasets, now at the census tract level, and ouput dataframe for modeling

In [33]:
social = (
    places
    .merge(census, on='geoid')
    .merge(walk, on='geoid')
    .merge(vote_ct[['geoid', 'cfb_votes']], on='geoid', how='left')
    .merge(crime_total[['geoid', 'nypd_violent']], on='geoid', how='left')
    .merge(ag_total[['geoid', 'nc_agriculture']], on='geoid', how='left')
    .merge(bldg_total, on='geoid', how='left')
).fillna(0)

social.to_csv('social.csv', index=False)

social.describe()

,cdc_physical,cdc_mental,population,acs_income,acs_participation,acs_gini,acs_housing,acs_uninsured,acs_bachelors,acs_hs,acs_white,acs_foreign,acs_alone,acs_child,acs_vacancy,acs_commute60,census_response,area,density,epa_walk,cfb_votes,nypd_violent,nc_agriculture,pluto_church,pluto_cultural,pluto_outdoor,pluto_school,pluto_vacant
count,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000,1.977000e+03,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000
mean,12.092109,12.923672,3850.776935,67233.815377,63.152045,0.462404,54.904654,8.308773,35.164131,81.566303,32.356795,37.618811,28.542001,31.640799,8.702193,28.789690,62.037785,3.590928e+06,0.001810,13.869044,521.835104,370.082954,4.154780,20.789074,2.256955,5.547294,4.779970,36.029337
std,3.436537,2.855676,1921.031115,33169.628607,8.918321,0.065392,13.325002,5.154003,20.461771,11.265062,29.320701,14.832653,12.159253,11.260798,6.131210,12.553966,8.659691,6.073919e+06,0.001246,1.911985,364.839669,349.885244,7.365344,14.082231,1.953336,5.191175,3.357969,18.987055
min,4.000000,6.200000,60.000000,9939.000000,8.550186,0.253500,0.000000,0.000000,2.957907,37.684053,0.000000,5.715141,0.000000,0.000000,0.000000,0.000000,21.400000,4.161252e+05,0.000002,6.666667,1.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10.000000,11.000000,2399.000000,44514.000000,57.975460,0.416900,46.322581,4.753381,20.212102,74.031621,4.365672,25.912409,19.457014,23.646209,4.905660,19.978225,55.900000,1.676125e+06,0.000909,12.583333,276.000000,133.666667,0.000000,10.000000,1.000000,2.000000,2.000000,23.000000
50%,11.700000,12.500000,3537.000000,63125.000000,63.313180,0.457100,55.041628,7.470651,30.003887,83.259494,23.529412,37.314488,27.246377,32.363636,7.566204,30.282486,62.300000,1.961303e+06,0.001555,14.000000,416.000000,239.333333,1.000000,18.000000,2.000000,4.000000,4.000000,33.000000
75%,13.800000,14.600000,4829.000000,83092.000000,68.164062,0.499300,63.692946,10.945008,45.159485,90.224033,58.414840,47.813411,35.540070,39.609994,11.027397,38.220481,68.700000,2.997727e+06,0.002393,15.166667,659.000000,523.333333,5.000000,29.000000,3.000000,8.000000,6.000000,46.000000
max,29.900000,24.000000,15456.000000,250001.000000,97.163121,0.750600,100.000000,45.087001,96.140082,100.000000,99.590723,82.991803,95.353535,84.244792,83.934426,70.534459,100.000000,1.170070e+08,0.009349,19.666667,2444.000000,2677.333333,51.000000,99.000000,13.000000,40.000000,19.000000,124.000000
